## This notebook was only created to test the ReadAndRunAllWorkFlows.py


<br>
<br>

In [1]:
#pull the ara results and look through KF+G edge graphs and pull the povenance attribute and make a table that will look like
#rows will be kp
#col be workflows
#entry would be the ara's that had edges in kp in the workflow. 

In [2]:
import requests
import json
from collections import defaultdict
import os
import pandas as pd
import numpy as np


#https://ars.ci.transltr.io/ars/api --- 
#https://arax.ci.transltr.io -- backup url

In [3]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

In [4]:
def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' 
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

<br>

### Submiting more than one jobs (test)

<br>

In [ ]:
urls = ['/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.9_EGFR_advanced.json', '/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json']

In [ ]:
dict_workflows = {}
for i in urls:
    feature = (os.path.splitext(os.path.basename(i))[0])
    print(i, feature)
    with open(i,'r') as inf:
        query = json.load(inf)
        kcresult = submit_to_ars(query)
        #xx = retrieve_devars_results(kcresult)
        
        dict_workflows[feature] = kcresult

In [ ]:
dict_workflows

In [ ]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

In [ ]:
workflow_result_messages

In [ ]:
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
        if key.startswith('kp-'):
            key_mod = key.replace('kp-','')
        else:
            key_mod = key
        
        final_dict[key_mod].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

In [ ]:
df

In [ ]:
df.loc['pk_id']

In [ ]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
pwd

<br>

## Test for individual run

<br>

In [ ]:
with open('/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.0_RHOBTB2_direct.json') as inf:
    query1 = json.load(inf)

In [ ]:
printjson(query1)

In [ ]:
kcresult = submit_to_ars(query1)

In [ ]:
#retrieve_ars_results(kcresult)

In [271]:
kcresult = '348dfa51-fa35-40b1-9225-7121897be0b3'
ars_url='https://ars.transltr.io/ars/api'
pk = 'https://arax.ncats.io/?source=ARS&id=' + kcresult
message_url = f'{ars_url}/messages/{kcresult}?trace=y'
response = requests.get(message_url)
j = response.json()

def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

#check_sheet.set_index('Workflow', inplace=True, drop=True)

In [272]:
if 'D.1_parkinsons-crohns.json' in list(check_sheet.Workflow):
    dfy = check_sheet[check_sheet['Workflow']== 'D.1_parkinsons-crohns.json']
    dfy.reset_index(drop=True)

    for index,val in enumerate(dfx.Curie):
        print(index,val)
        node_num = dfy.iloc[index][3]
        query_id = val
        if np.isnan(dfy.iloc[index][2]) == True:
            len_check = 2000
        else:
            len_check = dfy.iloc[index][2]

        print(index, query_id, node_num, len_check)

0 NCBIGene:120892
0 NCBIGene:120892 n01 2000
1 NCBIGene:11315
1 NCBIGene:11315 n01 2000
2 NCBIGene:110357
2 NCBIGene:110357 n01 2000


In [200]:
dfy

,Workflow,Curie,N (size of list of results),Query node ID,Include/Exclude,Unnamed: 5,Unnamed: 6
9,D.1_parkinsons-crohns.json,NCBIGene:120892,NaN,n01,include,NaN,LRRK2
10,D.1_parkinsons-crohns.json,NCBIGene:11315,NaN,n01,include,NaN,PARK7
11,D.1_parkinsons-crohns.json,NCBIGene:110357,NaN,n01,include,NaN,MOD2


In [205]:
if np.isnan(dfy.iloc[0][2]) == True:
    print('walla')

walla


In [166]:
check_sheet.columns

Index(['Workflow', 'Curie', 'N (size of list of results)', 'Query node ID',
       'Include/Exclude', 'Unnamed: 5', 'Unnamed: 6'],
      dtype='object')

In [276]:
results = {}
dictionary = {}
dictionary_2 = {}
dict3 = {}
for child in j['children']:
    print(child['status'])
    error_code = child['code']

    if child['status']  == 'Done':
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        try:
            
            child_response = requests.get(child_url).json()
            nresults = len(child_response['fields']['data']['message']['results'])
            
            if nresults > 0:
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                
            
            if 'D.1_parkinsons-crohns.json' in list(check_sheet.Workflow):
                dfy = check_sheet[check_sheet['Workflow']== 'D.1_parkinsons-crohns.json']
            
                dfy.reset_index(drop=True)
            
            

                for index,curie_id in enumerate(dfy.Curie):
                    print(index,curie_id)
                    node_num = dfy.iloc[index][3]
                    query_id = curie_id
                    if np.isnan(dfy.iloc[index][2]) == True:
                        len_check = len(child_response['fields']['data']['message']['results'])
                    else:
                        len_check = dfy.iloc[index][2]

                    #print(node_num, query_id, len_check)

                    locs = []
                    for x, val in enumerate(child_response['fields']['data']['message']['results']):
                        #print(val)

                        if x < len_check:

                            if query_id in val['node_bindings'][node_num][0]['id']:
                                locs.append(x)
                    if not locs:
                        check_result = f'False'
                        print(check_result)
                        #pass
                    else:
                        check_result = f'True'
                        print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                    dict3[curie_id] = check_result
            
            if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                        edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                        test_att_values =[]
                        for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            #print(val)

                            for tx in edge_ex[val]['attributes']:
                                if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :


                                    value_att = tx['value']

                                    test_att_values.append(value_att)
                                    test_att = set(flatten_list(test_att_values))


                                    dictionary_2[child['actor']['agent']] = test_att
                #else:
                    #dictionary_2[child['actor']['agent']] = [] 
            #else:
               # dictionary_2[child['actor']['agent']] = []

        except Exception as e:
            nresults=0
            child['status'] = 'ARS Error'
            #dictionary_2[child['actor']['agent']] = []



    elif child['status'] == 'Error':
        nresults=0
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        try:
            child_response = requests.get(child_url).json()
            results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            #dictionary_2[child['actor']['agent']] = []
        except Exception as e:
            #print(e)
            child['status'] = 'ARS Error'
            #dictionary_2[child['actor']['agent']] = []


    else:
        nresults = 0
        #dictionary_2[child['actor']['agent']] = []

    dictionary['pk_id'] =  pk  

    if ((child['status'] == 'Done') & (nresults == 0)):
        dictionary[child['actor']['agent']] = 'No Results: ' + str(error_code)
        #test =  [child['actor']['agent'], 'No Results']
    elif ((child['status'] == 'ARS Error') & (nresults == 0)):
        dictionary[child['actor']['agent']] = 'ARS Error: ' + str(error_code)
    elif ((child['status'] == 'Error') & (nresults == 0)):
        dictionary[child['actor']['agent']] = 'Error: ' + str(error_code)
        #test =  [child['actor']['agent'], 'ARS Error']
    elif ((child['status'] == 'Done') & (nresults != 0)):
        #test =  [child['actor']['agent'], 'Results']
        dictionary[child['actor']['agent']] = 'Results: ' + str(error_code) + str(dict3)
    elif ((child['status'] == 'Unknown') & (nresults == 0)):
        #test =  [child['actor']['agent'], 'Results']
        dictionary[child['actor']['agent']] = 'Unknown: ' + str(error_code) 

    print(child['actor']['agent'], child['status'], nresults)
    #test =  [child['actor']['agent'], child['status'], nresults]
    #test2.append(test)

Error
ara-aragorn-exp Error 0
Done
0 NCBIGene:120892
curie id: NCBIGene:120892 : INCLUDED at postion N == [0] on n01
1 NCBIGene:11315
curie id: NCBIGene:11315 : INCLUDED at postion N == [2] on n01
2 NCBIGene:110357
False
ara-improving Done 22
Error
kp-molecular Error 0
Done
0 NCBIGene:120892
curie id: NCBIGene:120892 : INCLUDED at postion N == [22] on n01
1 NCBIGene:11315
curie id: NCBIGene:11315 : INCLUDED at postion N == [28] on n01
2 NCBIGene:110357
False
ara-bte Done 394
Done
0 NCBIGene:120892
curie id: NCBIGene:120892 : INCLUDED at postion N == [1, 81, 82, 140, 183, 184, 288, 329, 330, 333, 334, 337, 340, 341, 403, 453, 491, 522, 683, 684, 687, 692, 812, 813, 814, 822, 1097, 1243, 1256, 1952, 1953, 1954, 1955] on n01
1 NCBIGene:11315
False
2 NCBIGene:110357
False
ara-aragorn Done 2140
Unknown
kp-genetics Unknown 0
Error
ara-robokop Error 0
Done
0 NCBIGene:120892
False
1 NCBIGene:11315
False
2 NCBIGene:110357
False
kp-openpredict Done 0
Done
0 NCBIGene:120892
False
1 NCBIGene:11315

In [274]:
dictionary_2

{'ara-improving': {'infores:chembl',
  'infores:diseases',
  'infores:gwas-catalog',
  'infores:improving-agent',
  'infores:mesh',
  'infores:spoke'},
 'ara-bte': {'infores:biolink-api',
  'infores:biothings-diseases',
  'infores:biothings-multiomics-clinical-risk',
  'infores:biothings-semmeddb-disease',
  'infores:chembl',
  'infores:clingen',
  'infores:ctd',
  'infores:diseases',
  'infores:disgenet',
  'infores:drugcentral',
  'infores:monarchinitiative',
  'infores:mondo',
  'infores:mychem-info',
  'infores:mydisease-info',
  'infores:mygene-info',
  'infores:semmeddb',
  'infores:translator-biothings-explorer'},
 'ara-aragorn': {'CTD',
  'DisGeNET',
  'JensenLab Knowledge GHR',
  'JensenLab Knowledge UniProtKB-KW',
  'JensenLab Text Mining',
  'Monarch',
  'UniProt Disease',
  'infores:aragorn',
  'infores:aragorn-ranker-ara',
  'infores:automat-biolink',
  'infores:automat-covidkop',
  'infores:automat-ctd',
  'infores:automat-drug-central',
  'infores:automat-hetio',
  'info

In [277]:
dictionary

{'pk_id': 'https://arax.ncats.io/?source=ARS&id=348dfa51-fa35-40b1-9225-7121897be0b3',
 'ara-aragorn-exp': 'Error: 404',
 'ara-improving': "Results: 200{'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}",
 'kp-molecular': 'Error: 501',
 'ara-bte': "Results: 200{'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}",
 'ara-aragorn': "Results: 200{'NCBIGene:120892': 'True', 'NCBIGene:11315': 'False', 'NCBIGene:110357': 'False'}",
 'kp-genetics': 'Unknown: 503',
 'ara-robokop': 'Error: 500',
 'kp-openpredict': 'No Results: 200',
 'kp-chp': 'No Results: 200',
 'ara-arax': 'Unknown: 503',
 'kp-cohd': 'Error: 400',
 'ara-ncats': 'Unknown: 503',
 'kp-icees': 'ARS Error: 200',
 'ara-explanatory': 'Error: 500',
 'ara-unsecret': 'Error: 504',
 'kp-cam': 'Error: 502',
 'kp-textmining': 'No Results: 200',
 'kp-icees-dili': 'ARS Error: 200'}

In [268]:
dict3

{'NCBIGene:120892': 'False',
 'NCBIGene:11315': 'False',
 'NCBIGene:110357': 'False'}

In [256]:
'test' + str(dict3)

"test{'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'True'}"

In [108]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *
import gspread_dataframe as gd


scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [131]:
ws = gc.open("workflow_progress_tracker").worksheet("Workflow Progress Tracker_2021_09_26-08_23_47_AM")

#existing = gd.get_as_dataframe(ws)
existing = pd.DataFrame(ws.get_all_records())

existing.set_index('', inplace=True)



existing = existing.rename_axis(None)

In [132]:
existing = existing[existing['Query Type']=='Sync']

In [133]:
existing

,Query Type,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,...,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.5_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Sync,No Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,...,Results: 200,Results: 200,Error: 500,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,Error: 598
ara-aragorn-exp,Sync,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,Sync,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,...,Results: 200,Unknown: 503,Error: 598,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-bte,Sync,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,...,Results: 200,Error: 598,Error: 598,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200
ara-explanatory,Sync,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,...,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-improving,Sync,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,...,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-ncats,Sync,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-robokop,Sync,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,...,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-unsecret,Sync,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,...,Results: 200,Error: 504,Error: 504,No Results: 200,Error: 504,Error: 504,Error: 504,Results: 200,No Results: 200,No Results: 200
kp-cam,Sync,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502


In [134]:
keep_same = {'Query Type'}
existing.columns = ['{}{}'.format(c, '' if c in keep_same else '.json') for c in existing.columns]

In [135]:
existing.columns

Index(['Query Type', 'A.0_RHOBTB2_direct.json', 'A.1_RHOBTB2.json',
       'A.2_RHOBTB2_twohop.json', 'A.2_RHOBTB2_twohop_constrained.json',
       'A.2a_RHOBTB2_twohop.json', 'A.2a_expanded_RHOBTB2_twohop.json',
       'A.2a_expanded_RHOBTB2_twohop_constrained.json', 'A.3_KCNMA1.json',
       'A.8_EGFR_simple.json', 'A.9_EGFR_advanced.json',
       'A.Multiomics_BigGIM_DR_KP_EGFR.json',
       'A.Multiomics_BigGIM_DR_KP_RHOBTB2.json',
       'B.1_DILI-three-hop-related-to.json',
       'B.2_DILI-three-hop-interacts-with.json',
       'B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone.json',
       'B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol.json',
       'B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin.json',
       'B.3d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin.json',
       'B.3e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate.json',
       'B.3f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone.json',
       'B.3g_DILI-fourth-one-

In [150]:
for i, indices in enumerate(check_sheet.Workflow):
    #if indices in existing.columns:
        print(indices,i)

A.0_RHOBTB2_direct.json 0
B.1a_DILI-three-hop-from-MONDO:0005359_DILI.json 1
B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis.json 2
B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis.json 3
B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis.json 4
B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI.json 5
B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI.json 6
B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI.json 7
B.1d_DILI_three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease.json 8
D.1_parkinsons-crohns.json 9
D.1_parkinsons-crohns.json 10
D.1_parkinsons-crohns.json 11
D.2_ssri-heart-disease.json 12
D.2_ssri-heart-disease.json 13
D.2_ssri-heart-disease.json 14
D.3_ssri-heart-disease-one-hop.json 15
D.4_tryptophan-kyurenine.json 16
D.4_tryptophan-kyurenine.json 17
D.4_tryptophan-kyurenine.json 18
D.4_tryptophan-kyurenine.json 19
D.6_metformin-ferritin.json 20
D.6_metformin-ferritin.json 21
D.6_metformin-ferri